In [1]:
import sys
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file1 = "stations.json.gz"
file2 = "city_data.csv"
output = "output.svg"

In [3]:
station_fh = gzip.open(file1, 'rt', encoding='utf-8')
stations = pd.read_json(station_fh, lines=True)
stations['avg_tmax'] = np.divide(stations['avg_tmax'],10)
print(stations)

       avg_tmax  elevation  latitude  longitude  observations      station
0     10.201667      691.0   54.4500  -124.2833           300  CA001092970
1     13.930937      967.0   50.0333  -113.2167           320  CA003030529
2     12.355311      902.0   49.0000  -108.3833           273  CA004038116
3     14.597727       16.0   46.5833   -72.2333           220  CA007016840
4     21.910429      190.8   34.5128   -93.0486           326  USC00033466
5     17.921788     1864.5   39.4286  -105.0703           358  USC00057249
6     25.847268       94.8   32.7128   -82.5414           366  USC00094862
7     15.010929      722.7   48.3511  -116.8353           366  USC00107386
8     14.479508     1501.1   43.8867  -111.7367           366  USC00108818
9      9.858356      185.9   47.2386   -68.6136           365  USC00172878
10    16.819399       10.7   42.2269   -70.9125           366  USC00193624
11    13.329714      318.8   44.6392   -85.3250           350  USC00207684
12    22.540548      105.

In [4]:
cities = pd.read_csv(file2).dropna(subset=["population", 'area'])
cities["area"] = np.divide(cities["area"], 1000000)
cities = cities[cities["area"] <= 10000]
cities["density"] = np.divide(cities["population"], cities["area"])
print(cities)

                 name  population         area   latitude   longitude  \
2             Calgary   1096833.0   825.290000  51.054444 -114.066944   
6            Edmonton    812201.0   684.370000  53.500000 -113.500000   
18         Abbotsford    133497.0   375.550000  49.054611 -122.328000   
20            Burnaby    223218.0    90.610000  49.250000 -122.949167   
42            Nanaimo     83811.0    91.000000  49.164167 -123.936389   
56         Revelstoke      7139.0    40.000000  50.998100 -118.196000   
57           Richmond    198309.0   129.270000  49.166667 -123.133333   
60             Surrey    468251.0   316.410000  49.183300 -122.850000   
63          Vancouver    603502.0   115.000000  49.250000 -123.100000   
65           Victoria     80032.0    19.470000  48.422151 -123.365700   
83        Fredericton     56224.0   130.680000  45.950000  -66.666667   
89         St. John's    106172.0   446.060000  47.567500  -52.707222   
91            Iqaluit      6699.0    52.000000  63.

In [5]:
def distance(city, stations):
    R2 = 12742000
    city_lat = np.deg2rad(city["latitude"])
    city_lon = np.deg2rad(city["longitude"])
    sta_lat = np.deg2rad(stations["latitude"])
    sta_lon = np.deg2rad(stations["longitude"])
    deltalat = np.deg2rad(sta_lat-city_lat)
    deltalon = np.deg2rad(sta_lon-city_lon)
    
    a = np.square(np.sin(deltalat/2)) + np.cos(np.deg2rad(city_lat))*np.cos(np.deg2rad(sta_lat))*np.square(np.sin(deltalon/2))
    return np.multiply(R2,np.arcsin(np.sqrt(a)))

In [6]:
def best_tmax(city, stations):
    dis = distance(city, stations)
    min_dis = np.argmin(dis)
    city["avg_tmax"] = stations.iloc[min_dis]["avg_tmax"]
    return city

In [7]:
cities = cities.apply(best_tmax, stations=stations, axis=1)

In [12]:
plt.plot(cities["avg_tmax"], cities["density"], 'b.', alpha=0.5)
plt.title("Temperature vs Population Density")
plt.xlabel("Avg Max Temperature (\u00b0C)")
plt.ylabel("Population Density (people/km\u00b2)")
plt.savefig(output)